# Imports

In [1]:
import copy
from multiprocessing import Pool, cpu_count

import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import cm
from matplotlib.ticker import LinearLocator
from qutip import *
from scipy.integrate import quad, simpson
from smallqnetnoiseML_functions import *
from tqdm.auto import tqdm

np.random.seed(1856543034)

# Parameters

## Hamiltonians under consideration
Diagonal Hamiltonian: $H_0 = \delta_p\ket{1}\bra{1} + \delta\ket{2}\bra{2}$\
Control Hamiltonian: $H_c = \frac{\Omega_p(t)}{2}(\ket{0}\bra{1}+ \ket{1}\bra{0}) + \frac{\Omega_s(t)}{2}(\ket{1}\bra{2} + \bra{2}\ket{1})$\
Noise Hamiltonian: $H_\mathrm{noise} = \tilde x_1(t)\ket{1}\bra{1} + \tilde x_2(t)\ket{2}\bra{2}$\
Total Hamiltonian: $H = H_0 + H_c + H_\mathrm{noise}$

In [7]:
T = 1
τ = 0.7 * T
N = 311
Ωp_max = 50 / T
Ωs_max = 50 / T
γ = 2.5 / T
t = np.linspace(-5, 5, 257) * T
t_noise = np.linspace(-5, 5, N) * T

η = 0.5

δ = 0
δp = 0

Δt = t_noise[1] - t_noise[0]

σ = 17.6068

# For Gaussian distribution
x1, x2 = -5 * σ, 5 * σ
xrange = np.linspace(x1, x2, 311)


# Tunneling rate between dot1 and dot2/Pump field
def Omega_p(t, args):
    return args["Ωp_max"] * np.exp(-(((t - args["τ"]) / args["T"]) ** 2))


# Tunneling rate between dot2 and dot3/Stokes field
def Omega_s(t, args):
    return args["Ωs_max"] * np.exp(-(((t + args["τ"]) / args["T"]) ** 2))


H0 = δp * e * e.dag() + δ * r * r.dag()  # Diagonal Hamiltonian
ψA0 = g  # Initial state of the system

if γ >= 0.0:
    c_ops = np.sqrt(γ) * (e * e.dag() + η * r * r.dag())  # Collapse operator = sqrt(γ)A

pars = {
    "Ωp_max": Ωp_max,
    "Ωs_max": Ωs_max,
    "τ": τ,
    "T": T,
    "ψA0": ψA0,
    "xrange": xrange,
    "N": N,
    "δ": δ,
    "δp": δp,
    "γ": γ,
}

# Data set generation

In [69]:
# Maximum tunnelling amplitudes under three conditions: Ωp_max = Ωs_max, Ωp_max > Ωs_max, Ωp_max < Ωs_max
Ωp_max = [50.0, 20 * np.sqrt(10), 10 * np.sqrt(10)]
Ωs_max = [50.0, 10 * np.sqrt(10), 20 * np.sqrt(10)]

num_trajectories = 225
num_samples = 500

## Data for correlated, non-Markovian quasistatic noise

In [9]:
# Range of correlation parameter η for correlated, non-Markovian quasistatic noise
ηrange_correlated = np.random.uniform(0.1, 5, num_samples)


def get_data_CN_wrapper(i):
    result = get_data_CN(
        t,
        gaussian,
        Omega_p,
        Omega_s,
        Ωp_max,
        Ωs_max,
        [ηrange_correlated[i]],
        num_trajectories,
        1,
        pars,
    )
    return result

In [10]:
data = []
with Pool(processes=cpu_count() - 6) as pool:
    for x in tqdm(
        pool.imap(get_data_CN_wrapper, np.arange(num_samples)), total=num_samples
    ):
        data += [x]

  0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
data1 = process_parallel_data(data)
efficiency_correlated, y_correlated, eta_CN = data1

In [13]:
# save_object(
#     [efficiency_correlated, y_correlated, eta_CN],
#     "../data/data_CN.pkl",
# )

In [14]:
efficiency_correlated, y_correlated, eta_CN = load_object(
    "../data/data_CN.pkl",
)

## Data for anti-correlated, non-Markovian quasistatic noise

In [15]:
# Range of correlation parameter η for correlated, non-Markovian quasistatic noise
ηrange_anti_correlated = np.random.uniform(-5, -0.1, num_samples)


def get_data_ACN_wrapper(i):
    result = get_data_ACN(
        t,
        gaussian,
        Omega_p,
        Omega_s,
        Ωp_max,
        Ωs_max,
        [ηrange_anti_correlated[i]],
        num_trajectories,
        1,
        pars,
    )
    return result

In [16]:
data = []
with Pool(processes=cpu_count() - 2) as pool:
    for x in tqdm(
        pool.imap(get_data_ACN_wrapper, np.arange(num_samples)), total=num_samples
    ):
        data += [x]

  0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
data2 = process_parallel_data(data)
efficiency_anti_correlated, y_anti_correlated, eta_ACN = data2

In [19]:
# save_object(
#     [efficiency_anti_correlated, y_anti_correlated, eta_ACN],
#     "../data/data_ACN.pkl",
# )

In [20]:
efficiency_anti_correlated, y_anti_correlated, eta_ACN = load_object(
    "../data/data_ACN.pkl",
)

## Data for uncorrelated, non-Markovian quasistatic noise

In [21]:
# Range of correlation parameter η for uncorrelated, non-Markovian quasistatic noise
ηrange_not_correlated = np.random.uniform(-5, 5, num_samples)


def get_data_UCN_wrapper(i):
    result = get_data_UCN(
        t,
        gaussian_2d,
        Omega_p,
        Omega_s,
        Ωp_max,
        Ωs_max,
        [ηrange_not_correlated[i]],
        num_trajectories,
        1,
        pars,
    )
    return result

In [22]:
data = []
with Pool(processes=cpu_count() - 6) as pool:
    for x in tqdm(
        pool.imap(get_data_UCN_wrapper, np.arange(num_samples)), total=num_samples
    ):
        data += [x]

  0%|          | 0/50 [00:00<?, ?it/s]

In [27]:
data3 = process_parallel_data(data)
efficiency_uncorrelated, y_uncorrelated, eta_UCN = data3

In [28]:
# save_object(
#     [efficiency_uncorrelated, y_uncorrelated, eta_UCN],
#     "../data/data_UCN.pkl",
# )

In [29]:
efficiency_uncorrelated, y_uncorrelated, eta_UCN = load_object(
    "../data/data_UCN.pkl",
)

## Data for Markovian (correlated and anticorrelated) noise

In [30]:
# Range of correlation parameter η for Markovian noise
# For the case of correlated noise, change the range as np.random.uniform(0.1, 5, num_samples)"
# For the case of anti-correlated noise, change the range as np.random.uniform(-5, -0.1, num_samples)"
ηrange_markovian = np.random.uniform(-5, 5, num_samples)


def get_data_NQS_wrapper(i):
    "Remember to change 'separated = True' if correlated or anti-correlated case is considered!!!"
    result = get_data_NQS(
        t,
        Omega_p,
        Omega_s,
        Ωp_max,
        Ωs_max,
        [ηrange_markovian[i]],
        1,
        pars,
        separated=False,
    )
    return result

In [31]:
data = []
with Pool(processes=cpu_count() - 8) as pool:
    for x in tqdm(
        pool.imap(get_data_NQS_wrapper, np.arange(num_samples)), total=num_samples
    ):
        data += [x]

  0%|          | 0/50 [00:00<?, ?it/s]

In [32]:
data6 = process_parallel_data(data)
efficiency_markovian, y_markovian, eta_M = data6

In [33]:
# save_object(
#     [efficiency_markovian, y_markovian, eta_M],
#     "../data/data_CM.pkl",
# )

In [34]:
efficiency_markovian, y_markovian, eta_M = load_object(
    "../data/data_CM.pkl",
)

# Calculation with finite number of projective measurement

In [68]:
# Maximum tunnelling amplitudes under three conditions: Ωp_max = Ωs_max, Ωp_max > Ωs_max, Ωp_max < Ωs_max (same as before)
Ωp_max = [50.0, 20 * np.sqrt(10), 10 * np.sqrt(10)]
Ωs_max = [50.0, 10 * np.sqrt(10), 20 * np.sqrt(10)]

# We generate each sample of data by averaging over 5k peojective measurements
num_trajectories = 5000
num_samples = 500

## Data for correlated, non-Markovian quasistatic noise

In [40]:
# Range of correlation parameter η for correlated, non-Markovian quasistatic noise
ηrange_correlated = np.random.uniform(0.1, 5, num_samples)


# Generation of data with multiprocessing unit
def get_data_CN_finite_wrapper(i):
    result = get_data_CN_finite(
        t,
        t_noise,
        gaussian,
        Omega_p,
        Omega_s,
        Ωp_max,
        Ωs_max,
        [ηrange_correlated[i]],
        num_trajectories,
        1,
        pars,
    )
    return result

In [41]:
data = []
with Pool(processes=cpu_count() - 6) as pool:
    for x in tqdm(
        pool.imap(get_data_CN_finite_wrapper, np.arange(num_samples)), total=num_samples
    ):
        data += [x]

  0%|          | 0/50 [00:00<?, ?it/s]

In [44]:
data1 = process_parallel_data(data)
efficiency_correlated, y_correlated, eta_CN, S_CN = data1

In [45]:
# save_object(
#     [efficiency_correlated, y_correlated, eta_CN, S_CN],
#     "../data/dataE_CN.pkl",
# )

In [46]:
efficiency_correlated, y_correlated, eta_CN, S_CN = load_object("../data/dataE_CN.pkl")

In [47]:
np.array(S_CN).shape

(50, 3, 50)

## Data for anti-correlated, non-Markovian quasistatic noise

In [48]:
# Range of correlation parameter η for correlated, non-Markovian quasistatic noise
ηrange_anti_correlated = np.random.uniform(-5, -0.1, num_samples)


def get_data_ACN_finite_wrapper(i):
    result = get_data_ACN_finite(
        t,
        t_noise,
        gaussian,
        Omega_p,
        Omega_s,
        Ωp_max,
        Ωs_max,
        [ηrange_anti_correlated[i]],
        num_trajectories,
        1,
        pars,
    )
    return result

In [49]:
data = []
with Pool(processes=cpu_count() - 6) as pool:
    for x in tqdm(
        pool.imap(get_data_ACN_finite_wrapper, np.arange(num_samples)),
        total=num_samples,
    ):
        data += [x]

  0%|          | 0/50 [00:00<?, ?it/s]

In [50]:
data2 = process_parallel_data(data)
efficiency_anti_correlated, y_anti_correlated, eta_ACN, S_ACN = data2

In [51]:
# save_object(
#     [efficiency_anti_correlated, y_anti_correlated, eta_ACN, S_ACN],
#     "../data/dataE_ACN.pkl",
# )

In [52]:
efficiency_anti_correlated, y_anti_correlated, eta_ACN, S_ACN = load_object(
    "../data/dataE_ACN.pkl"
)

In [53]:
np.array(S_ACN).shape

(50, 3, 50)

## Data for uncorrelated, non-Markovian quasistatic noise

In [54]:
# Range of correlation parameter η for uncorrelated, non-Markovian quasistatic noise
ηrange_not_correlated = np.random.uniform(-5, 5, num_samples)


def get_data_UCN_finite_wrapper(i):
    result = get_data_UCN_finite(
        t,
        t_noise,
        gaussian,
        Omega_p,
        Omega_s,
        Ωp_max,
        Ωs_max,
        [ηrange_not_correlated[i]],
        num_trajectories,
        1,
        pars,
    )
    return result

In [55]:
data = []
with Pool(processes=cpu_count() - 6) as pool:
    for x in tqdm(
        pool.imap(get_data_UCN_finite_wrapper, np.arange(num_samples)),
        total=num_samples,
    ):
        data += [x]

  0%|          | 0/50 [00:00<?, ?it/s]

In [56]:
data3 = process_parallel_data(data)
efficiency_uncorrelated, y_uncorrelated, eta_UCN, S_UCN = data3

In [57]:
# save_object(
#     [efficiency_uncorrelated, y_uncorrelated, eta_UCN, S_UCN],
#     "../data/dataE_UCN.pkl",
# )

In [60]:
efficiency_uncorrelated, y_uncorrelated, eta_UCN, S_UCN = load_object(
    "../data/dataE_UCN.pkl"
)

In [61]:
np.array(S_UCN).shape

(50, 3, 50)

## Data for Markovian noise

In [62]:
# Range of correlation parameter η for Markovian noise
ηrange_nonquasistatic = np.random.uniform(-5, 5, num_samples)


def get_data_NQS_finite_wrapper(i):
    result = get_data_NQS_finite(
        t,
        t_noise,
        gaussian,
        Omega_p,
        Omega_s,
        Ωp_max,
        Ωs_max,
        [ηrange_nonquasistatic[i]],
        num_trajectories,
        1,
        pars,
    )
    return result

In [63]:
data = []
with Pool(processes=cpu_count() - 6) as pool:
    for x in tqdm(
        pool.imap(get_data_NQS_finite_wrapper, np.arange(num_samples)),
        total=num_samples,
    ):
        data += [x]

  0%|          | 0/50 [00:00<?, ?it/s]

In [64]:
data4 = process_parallel_data(data)
efficiency_non_quasistaic, y_non_quasistaic, eta_NQS, S_NQS = data4

In [65]:
# save_object(
#     [efficiency_non_quasistaic, y_non_quasistaic, eta_NQS, S_NQS],
#     "../data/dataE_M.pkl",
# )

In [66]:
efficiency_non_quasistaic, y_non_quasistaic, eta_NQS, S_NQS = load_object(
    "../data/dataE_M.pkl"
)